<a href="https://colab.research.google.com/github/asigalov61/Noobiano/blob/master/Noobiano.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Noobiano

## An introduction to Music Artificial Intelligence

### Begin your journey here. See what YOU can do and how it will compare to the State Of The Art Music Artificial Intelligence Systems.

This is a working fork from here:
https://github.com/JakeNims1305/DataScienceMusic

Huge thanks and respect for creating this colab goes out to:

Jake Nimergood jtn796, Alexander Issa api236, Michael Herrington mah6449, Anushree Biradar ajb5277, Isabelle Rogers , Arjun Singh

#Setup the Environment and Import all needed modules/dependencies

In [ ]:
#@title Install Keras and Import all Modules
#Run version 2.1.6
!pip install keras_self_attention

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

import glob
import pickle

from music21 import converter, instrument, stream, note, chord

from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Activation, Bidirectional, Flatten
from keras import utils
from keras.callbacks import ModelCheckpoint
from keras_self_attention import SeqSelfAttention

#Create a simple Bi-Directional LSTMs AI model.

##Define necessary functions

In [ ]:
#@title Main Model Training Function Definition
def train_network(notes, n_vocab):
    """ Train a Neural Network to generate music """
    network_input, network_output = prepare_sequences(notes, n_vocab)

    model = create_network(network_input, n_vocab)
    train(model, network_input, network_output)

Before running the cells below, upload your MIDI DataSet to the root folder of Colab (/content)

Here is one of my MIDIs DataSets you can try easily and right away if you do not have your own MIDIs (DataSet).

In [ ]:
#@title Alex Piano Only Original 450 MIDIs 
%cd /content/
!wget 'https://github.com/asigalov61/AlexMIDIDataSet/raw/master/AlexMIDIDataSet-CC-BY-NC-SA-Piano-Only.zip'
!unzip -j 'AlexMIDIDataSet-CC-BY-NC-SA-Piano-Only.zip'

In [ ]:
#@title Get all the notes and chords from the midi files
def get_notes():
    """ Get all the notes and chords from the midi files in the ./content directory. Called BEFORE train """
    notes = []
    durations = []

    for file in glob.glob("*.mid"):
        midi = converter.parse(file)

        print("Parsing %s" % file)

        notes_to_parse = None

        try: # file has instrument parts
            s2 = instrument.partitionByInstrument(midi, instrument.Piano) #Change to only grab the piano???
            notes_to_parse = s2.parts[0].recurse() 
        except: # file has notes in a flat structure
            notes_to_parse = midi.flat.notes

        for element in notes_to_parse:
            if isinstance(element, note.Note):
                notes.append(str(element.pitch) + " " +  str(element.quarterLength))
            elif isinstance(element, chord.Chord):
                notes.append('.'.join(str(n) for n in element.normalOrder) + " " + str(element.quarterLength))
            elif isinstance(element, note.Rest):
                notes.append(str(element.name)  + " " + str(element.quarterLength))

    with open('notes', 'wb') as filepath:
        pickle.dump(notes, filepath)

    return notes

In [ ]:
#@title Prepare the sequences used by the Neural Network
model_sequence_length = 64 #@param {type:"slider", min:1, max:512, step:1}
def prepare_sequences(notes, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    sequence_length = model_sequence_length

    # get all pitch names
    pitchnames = sorted(set(item for item in notes))

     # create a dictionary to map pitches to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    network_input = network_input / float(n_vocab)

    network_output = utils.to_categorical(network_output)

    return (network_input, network_output)

## Create Model Structure (3 Layer Bi-Directional LSTM with Softmax Activation and with Cross-Entropy Loss Function

In [ ]:
#@title Most Interesting Part: Create the structure of the neural network. You can easily add or remove layers to your liking in the code (see commented sections). So go ahead and have fun/let your imagination go wild. See if you can beat MuseNet or AIVA architecture and see how # of layers (types) affects the output of the model. :)

def create_network(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(Bidirectional(LSTM(512,
        input_shape=(network_input.shape[1], network_input.shape[2]), #n_time_steps, n_features?
        return_sequences=True)))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(Dropout(0.3))
    
    model.add(LSTM(512,return_sequences=True))
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Flatten()) #Supposedly needed to fix stuff before dense layer
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    return model

In [ ]:
#@title Main Training Function and Important Training Settings
number_of_training_epochs = 16 #@param {type:"slider", min:1, max:500, step:1}
training_batches_number = 16 #@param {type:"slider", min:1, max:32, step:1}
save_every_number_of_epochs = 15 #@param {type:"slider", min:1, max:100, step:1}
save_only_best_checkpoints = True #@param {type:"boolean"}
def train(model, network_input, network_output):
    """ train the neural network """
    filepath = os.path.abspath("weights-3LSTMAttLayer-{epoch:03d}-{loss:.4f}.hdf5")
    checkpoint = ModelCheckpoint(
        filepath,
        save_freq=save_every_number_of_epochs, #Every # epochs
        monitor='loss',
        verbose=1,
        save_best_only=save_only_best_checkpoints,
        mode='min'
    )
    callbacks_list = [checkpoint]

    model.fit(network_input, network_output, epochs=number_of_training_epochs, batch_size=training_batches_number, callbacks=callbacks_list)
    model.summary()

In [ ]:
#@title Load the notes ("notes" file) used to train the model if you have one. Skip this step if you do not.
def loadnotes(notes, n_vocab):

    with open('/content/notes', 'rb') as pickle_file:
         notes = pickle.load(pickle_file)

    # Get all pitch names
    n_vocab = len(set(notes))
 
    prepare_sequences(notes, n_vocab)

 notes = []
 n_vocab = []
 loadnotes(notes, n_vocab)

In [ ]:
#@title Parse all MIDI files into "notes" file for training/generation. Please note that this may take a long time if you uploaded a lot of files and because this is a very basic MIDI processor that was not optimized for speed/perfomance. Please also note that only notes info being saved and dynamics, as well as other important structural elements of music are NOT included in this processor. Please see my other colabs for more advanced MIDI processor code/implementation.

#load files in
notes = get_notes()

# get amount of pitch names
n_vocab = len(set(notes))

#Train your model here. This may also take a while depending on many different factors.

In [ ]:
#@title Start Training
#train
train_network(notes, n_vocab)

# Predict 

In [ ]:
def generate():
    """ Generate a piano midi file """
    #load the notes used to train the model
    with open('notes', 'rb') as filepath:
        notes = pickle.load(filepath)

    # Get all pitch names
    pitchnames = sorted(set(item for item in notes))
    # Get all pitch names
    n_vocab = len(set(notes))

    network_input, normalized_input = prepare_sequences_output(notes, pitchnames, n_vocab)
    model = create_network_add_weights(normalized_input, n_vocab)
    prediction_output = generate_notes(model, network_input, pitchnames, n_vocab)
    create_midi(prediction_output)

In [ ]:
def prepare_sequences_output(notes, pitchnames, n_vocab):
    """ Prepare the sequences used by the Neural Network """
    # map between notes and integers and back
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))

    sequence_length = 64
    network_input = []
    output = []
    for i in range(0, len(notes) - sequence_length, 1):
        sequence_in = notes[i:i + sequence_length]
        sequence_out = notes[i + sequence_length]
        network_input.append([note_to_int[char] for char in sequence_in])
        output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # reshape the input into a format compatible with LSTM layers
    normalized_input = np.reshape(network_input, (n_patterns, sequence_length, 1))
    # normalize input
    normalized_input = normalized_input / float(n_vocab)

    return (network_input, normalized_input)

In [ ]:
def create_network_add_weights(network_input, n_vocab):
    """ create the structure of the neural network """
    model = Sequential()
    model.add(Bidirectional(LSTM(512,
        input_shape=(network_input.shape[1], network_input.shape[2]), #n_time_steps, n_features?
        return_sequences=True)))
    model.add(SeqSelfAttention(attention_activation='sigmoid'))
    model.add(Dropout(0.3))
    
    model.add(LSTM(512,return_sequences=True))
    model.add(Dropout(0.3))
    
    model.add(Flatten()) #Supposedly needed to fix stuff before dense layer
    model.add(Dense(n_vocab))
    model.add(Activation('softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop')

    # Load the weights to each node
    model.load_weights('/content/weights-3LSTMAttLayer-001-8.5167.hdf5')
    
    return model

In [ ]:
def generate_notes(model, network_input, pitchnames, n_vocab):
    """ Generate notes from the neural network based on a sequence of notes """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(network_input)-1)

    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))

    pattern = network_input[start]
    prediction_output = []

    # generate 500 notes
    for note_index in range(1024):
        prediction_input = np.reshape(pattern, (1, len(pattern), 1))
        prediction_input = prediction_input / float(n_vocab)

        prediction = model.predict(prediction_input, verbose=0)

        index = np.argmax(prediction)
        result = int_to_note[index]
        prediction_output.append(result)

        pattern.append(index)
        pattern = pattern[1:len(pattern)]

    return prediction_output

In [ ]:
def create_midi(prediction_output):
    """ convert the output from the prediction to notes and create a midi file
        from the notes """
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        pattern = pattern.split()
        temp = pattern[0]
        duration = pattern[1]
        pattern = temp
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a rest
        elif('rest' in pattern):
            new_rest = note.Rest(pattern)
            new_rest.offset = offset
            new_rest.storedInstrument = instrument.Piano() #???
            output_notes.append(new_rest)
        # pattern is a note
        else:
            new_note = note.Note(pattern)
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)
        # increase offset each iteration so that notes do not stack
        offset += convert_to_float(duration)

    midi_stream = stream.Stream(output_notes)

    midi_stream.write('midi', fp='test_output.mid')
 
#From: https://stackoverflow.com/questions/1806278/convert-fraction-to-float
def convert_to_float(frac_str):
    try:
        return float(frac_str)
    except ValueError:
        num, denom = frac_str.split('/')
        try:
            leading, num = num.split(' ')
            whole = float(leading)
        except ValueError:
            whole = 0
        frac = float(num) / float(denom)
        return whole - frac if whole < 0 else whole + frac

In [ ]:
#RUN THE GENERATOR

generate()

In [ ]:
#Test stuff

with open('notes', 'rb') as filepath:
    notes = pickle.load(filepath)

# Get all pitch names
pitchnames = sorted(set(item for item in notes))
# Get all pitch names
n_vocab = len(set(notes))
    
print(pitchnames)
print(n_vocab)
print(notes)
print(len(notes))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')